In [ ]:
import cv2
import numpy as np

def color_based_detection(frame, target_color=(0, 0, 255)):  # Target color is blue by default
  """
  This function performs a simple color-based object detection in the frame.

  Args:
      frame: The current video frame as a NumPy array.
      target_color: The target color to detect in BGR format (default is blue).

  Returns:
      A list of bounding boxes (startX, startY, endX, endY) for detected objects.
  """

  # Convert the frame to HSV color space for better color detection
  hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

  # Define the color range for the target color (adjust these values if needed)
  lower_color = np.array([target_color[0] - 10, target_color[1] - 10, target_color[2] - 10])
  upper_color = np.array([target_color[0] + 10, target_color[1] + 10, target_color[2] + 10])

  # Create a mask to isolate pixels within the target color range
  mask = cv2.inRange(hsv, lower_color, upper_color)

  # Find contours (connected regions) in the mask
  contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

  # Initialize an empty list to store object bounding boxes
  detected_objects = []

  # Loop over the contours
  for cnt in contours:
    # Approximate the contour with a bounding rectangle
    x, y, w, h = cv2.boundingRect(cnt)

    # Append the bounding box to the list
    detected_objects.append((x, y, x + w, y + h))

  # Return the list of detected object bounding boxes
  return detected_objects

def centroid_tracker(objects, frame):
  """
  This function takes a list of objects (bounding boxes) and the frame as input.
  It calculates the centroid of each object and draws a circle around it.

  Args:
      objects: List of bounding boxes (startX, startY, endX, endY) for detected objects.
      frame: The current video frame as a NumPy array.

  Returns:
      The original frame with circles drawn around the centroids of detected objects.
  """

  # Initialize a list of colors for object tracking (randomly generated)
  colors = np.random.randint(0, 255, size=((len(objects), 3)), dtype="uint8")

  # Loop over the detected objects
  for i, (startX, startY, endX, endY) in enumerate(objects):
    # Calculate the centroid (center) of the bounding box
    cX = int((startX + endX) / 2)
    cY = int((startY + endY) / 2)

    # Draw a circle around the centroid with thickness 2 and random color
    cv2.circle(frame, (cX, cY), 2, colors[i].tolist(), -1)  # -1 for filled circle

  # Return the frame with circles drawn
  return frame

# Load the video (replace with your video path)
# video_path = 'path/to/your/video.mp4'
cap = cv2.VideoCapture(0) 

# Check if video was opened successfully
if not cap.isOpened():
  print("Error opening video file!")
  exit()

while True:
  # Read a frame from the video
  ret, frame = cap.read()

  # Check if frame was read successfully
  if not ret:
    break

  # Object detection using color_based_detection function
  objects = color_based_detection(frame)

  # Update trackers (not used in centroid tracking)
  tracker = None  # Not needed for centroid tracking

  # Use the centroid_tracker function to draw circles around object centroids
  tracked_frame = centroid_tracker(objects, frame.copy())  # Use a copy to avoid modifying original frame

  # Display the frame with tracked objects
  cv2.imshow("Object Tracking (Centroid)", tracked_frame)

  # Wait for a key press to close the window (e.g., 'q' for quit)
  key = cv2.waitKey(1)
